In [1]:
import sys

!which {sys.executable}

/ut3/jerome/miniconda3/envs/wakai/bin/python


In [2]:
import sys
sys.path.insert(0, '../code')

# NExSci catalog

In [4]:
from catalog import get_nexsci_data

nexsci = get_nexsci_data(clobber=False)
nexsci.tail()

Column definitions:  https://exoplanetarchive.ipac.caltech.edu/docs/API_PS_columns.html


UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.") [astropy.table.table]
UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.") [astropy.table.table]
UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.") [astropy.table.table]
UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.") [astropy.table.table]
UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.") [astropy.table.table]
UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.") [astropy.table.table]


Saved:  ../data/nexsci_ps.csv


,pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,gaia_id,default_flag,pl_refname,sy_refname,...,sy_jmagstr,sy_hmag,sy_hmagerr1,sy_hmagerr2,sy_hmagstr,sy_kmag,sy_kmagerr1,sy_kmagerr2,sky_coord.ra,sky_coord.dec
32872,TOI-237 b,b,TOI-237,,,TIC 305048087,Gaia DR2 2329387852426700800,0,<a refstr=EXOFOP_TESS_TOI href=https://exofop....,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,...,11.740&plusmn;0.021,11.190,0.022,-0.022,11.190&plusmn;0.022,10.896,0.025,-0.025,353.243538,-29.416487
32873,HD 1397 b,b,HD 1397,HD 1397,HIP 1419,TIC 394137592,Gaia DR2 4707634458245031552,0,<a refstr=EXOFOP_TESS_TOI href=https://exofop....,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,...,6.442&plusmn;0.020,6.090,0.049,-0.049,6.090&plusmn;0.049,5.988,0.018,-0.018,4.447163,-66.358929
32874,OGLE2-TR-L9 b,b,OGLE2-TR-L9,,,TIC 466840711,Gaia DR2 5337273743301313536,1,<a refstr=SNELLEN_ET_AL__2009 href=https://ui....,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,...,13.508&plusmn;0.032,13.246,0.035,-0.035,13.246&plusmn;0.035,13.117,0.037,-0.037,166.979879,-61.146250
32875,CoRoT-12 b,b,CoRoT-12,,,TIC 280210963,Gaia DR2 3107255598526406912,0,<a refstr=BONOMO_ET_AL__2017 href=https://ui.a...,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,...,14.024&plusmn;0.032,13.630,0.031,-0.031,13.630&plusmn;0.031,13.557,0.042,-0.042,100.765687,-1.296444
32876,CoRoT-20 b,b,CoRoT-20,,,TIC 234825296,Gaia DR2 3120122805143587328,0,<a refstr=BONOMO_ET_AL__2017 href=https://ui.a...,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,...,12.991&plusmn;0.026,12.652,0.027,-0.027,12.652&plusmn;0.027,12.513,0.029,-0.029,97.720425,0.226893


In [5]:
nexsci.shape

(32811, 356)

In [6]:
nexsci.hostname.unique().shape

(3244,)

# cross-match nexsci with Gaia DR3 using `astroquery.xmatch`

In [19]:
import astropy.units as u
from astroquery.xmatch import XMatch
import pandas as pd

gaia_table = XMatch.query(
    cat1=open('../data/nexsci_ps.csv'),
    cat2="vizier:I/355/gaiadr3",  # The Vizier identifier for Gaia DR3
    max_distance=1 * u.arcsec,    # Set the maximum matching radius
    colRA1="ra",  # Column name for Right Ascension in TIC results
    colDec1="dec" # Column name for Declination in TIC results
)
gaia_df = gaia_table.to_pandas()

# results to 6237 matches
matches = gaia_df.copy()

# Apply magnitude filtering criteria
# For main-sequence stars, the difference between TESS T and Gaia G is around 0–0.5 magnitudes
# Redder stars might have T - G values closer to 1–2 magnitudes.
# For giants or highly reddened stars, the difference could be larger.
if False:
    # results to 5971 matches
    matches = []
    for i, row in gaia_df.iterrows():
        teff=row['st_teff']
        if teff<3500:
            magnitude_threshold = 2
        elif (teff>3500) and (teff<6000):
            magnitude_threshold = 1
        else:
            magnitude_threshold = 1.5
        if abs(row['Gmag'] - row['TESS Mag']) < magnitude_threshold:
            matches.append(row)
    matches = pd.concat(matches, axis=1).T
if False:
    # results to 5700 matches
    magnitude_threshold = 1
    matches = gaia_df[abs(gaia_df['Gmag'] - gaia_df['sy_tmag']) < magnitude_threshold]
if False:
    # results to 4878 matches
    teff_threshold = 1000
    matches = gaia_df[abs(gaia_df['Teff'] - gaia_df['st_teff']) < teff_threshold]
matches.head()

Could not import regions, which is required for some of the functionalities of this module.


,angDist,pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,gaia_id,default_flag,pl_refname,...,B_AG,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
0,0.002697,OGLE-TR-10 b,b,OGLE-TR-10,NaN,NaN,TIC 130150682,Gaia DR2 4056443366649948160,1,<a refstr=TORRES_ET_AL__2008 href=https://ui.a...,...,1.0106,0.5408,0.5354,0.5464,MARCS,267.867747,-29.876453,1.044436,0.677822,0.6647
1,0.008326,KOI-13 b,b,KOI-13,NaN,NaN,TIC 158324245,Gaia DR2 2130632159134827392,0,<a refstr=BORUCKI_ET_AL__2011 href=https://ui....,...,NaN,NaN,NaN,NaN,NaN,286.971415,46.868312,0.684921,0.797563,0.0435
2,0.004001,Kepler-44 b,b,Kepler-44,NaN,NaN,TIC 239277658,Gaia DR2 2085222191474854656,0,<a refstr=BONOMO_ET_AL__2011 href=https://ui.a...,...,0.3457,0.1774,0.1709,0.1831,PHOENIX,300.102357,45.762156,0.287152,0.306157,0.0530
3,0.008326,KOI-13 b,b,KOI-13,NaN,NaN,TIC 158324245,Gaia DR2 2130632159134827392,0,<a refstr=BATALHA_ET_AL__2013 href=https://ui....,...,NaN,NaN,NaN,NaN,NaN,286.971415,46.868312,0.684921,0.797563,0.0435
4,0.015032,WASP-62 b,b,WASP-62,NaN,NaN,TIC 149603524,Gaia DR2 4756649415309914240,0,<a refstr=HELLIER_ET_AL__2012 href=https://ui....,...,0.0023,0.0005,0.0001,0.0013,MARCS,87.139972,-63.988441,0.270854,0.243980,-0.2165


In [22]:
matches.shape

(33088, 494)

In [23]:
matches.hostname.unique().shape

(3242,)

In [24]:
matches['GaiaDR3'] = matches['DR3Name'].apply(lambda x: int(x.split()[-1]))
matches.to_csv('../data/nexsci_GaiaDR3_xmatch.csv', index=False)

In [25]:
idx = nexsci.hostname.isin(matches.hostname.unique())
nexsci[~idx].shape

(10, 356)

In [26]:
nexsci[~idx].to_csv('../data/nexsci_GaiaDR3_xmatch_missing.csv', index=False)